In [16]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('./train.csv', encoding='big5')

In [17]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
print(data)
print(raw_data[0:2, :])

         0     1     2     3     4     5     6     7     8     9  ...    14  \
0       14    14    14    13    12    12    12    12    15    17  ...    22   
1      1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8  ...   1.8   
2     0.51  0.41  0.39  0.37  0.35   0.3  0.37  0.47  0.78  0.74  ...  0.37   
3      0.2  0.15  0.13  0.12  0.11  0.06   0.1  0.13  0.26  0.23  ...   0.1   
4      0.9   0.6   0.5   1.7   1.8   1.5   1.9   2.2   6.6   7.9  ...   2.5   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4315   1.8   1.8   1.8   1.8   1.8   1.7   1.7   1.8   1.8   1.8  ...   1.8   
4316    46    13    61    44    55    68    66    70    66    85  ...    59   
4317    36    55    72   327    74    52    59    83   106   105  ...    18   
4318   1.9   2.4   1.9   2.8   2.3   1.9   2.1   3.7   2.8   3.8  ...   2.3   
4319   0.7   0.8   1.8     1   1.9   1.7   2.1     2     2   1.7  ...   1.3   

        15    16    17    18    19    20    21    2

In [18]:
month_data = {}
for month in range(12):
    sample = np.empty([18,480])
    for day in range(20):
        sample[:, day*24 : (day+1)*24] = raw_data[18*(month*20+day) : 18*(month*20+day+1), :]
    month_data[month] = sample

In [35]:
x = np.empty([12*471, 18*9], dtype=float)
y = np.empty([12*471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day==19 and hour>14:
                continue
            x[month*471 + day*24 + hour, :] = month_data[month][:, day*24 + hour: day*24 + hour + 9].reshape(1,-1)
            y[month*471 + day*24 + hour, 0] = month_data[month][9, day*24 + hour + 9]
print(x.shape)
print(y.shape)

(5652, 162)
(5652, 1)


In [36]:
mean_x = np.mean(x, axis=0)
std_x = np.std(x, axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j]!=0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [37]:
import math
x_train_set = x[: math.floor(len(x)*0.8), :]
y_train_set = y[: math.floor(len(y)*0.8), :]
x_validation = x[math.floor(len(x)*0.8): , :]
y_validation = y[math.floor(len(x)*0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [38]:
dim = 18*9 + 1
w = np.zeros([dim,1])
x = np.concatenate((np.ones([12*471,1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 2000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001

for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w)-y, 2)) / 471 / 12)
    if t%100 == 0:
        print(str(t) + ":" + str(loss))
    
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient ** 2
    w = w - learning_rate*gradient/np.sqrt(adagrad+eps)
    
np.save('weight.npy', w)
w

0:27.071214829194115
100:33.78905859777454
200:19.9137512981971
300:13.531068193689693
400:10.645466158446172
500:9.277353455475067
600:8.518042045956502
700:8.014061987588422
800:7.63675682477569
900:7.3365637403711235
1000:7.0909686439472175
1100:6.887311480324098
1200:6.717116295730693
1300:6.574102121171867
1400:6.453381172520267
1500:6.351062466046933
1600:6.264012287766369
1700:6.1896883834532055
1800:6.126016764546804
1900:6.071297135545988


array([[ 2.13740269e+01],
       [-3.85358337e-01],
       [ 1.74238580e+00],
       [ 2.42857568e+00],
       [-1.31376283e+00],
       [-1.66050659e+01],
       [ 1.74945157e+01],
       [-3.25305868e-01],
       [-3.83030971e+00],
       [ 6.64382757e-01],
       [-3.06244747e-01],
       [-1.61136573e-02],
       [ 9.49548607e-02],
       [ 9.09413380e-02],
       [ 1.68254055e-01],
       [-1.22278191e-01],
       [-2.14629673e-01],
       [ 9.72877482e-02],
       [ 5.97408636e-01],
       [ 8.77781481e-02],
       [-3.13652231e-02],
       [ 5.58403947e-02],
       [-1.70307420e-01],
       [ 1.55512409e-01],
       [-4.68897995e-02],
       [-1.07102292e-01],
       [ 6.19063691e-02],
       [ 3.94938554e-01],
       [-1.19882458e-01],
       [ 1.59752321e-01],
       [-4.31373821e-01],
       [ 3.76837705e-01],
       [ 6.59273103e-01],
       [-8.00451212e-01],
       [ 3.88213797e-02],
       [ 1.88874097e-01],
       [ 1.90949068e-01],
       [-1.94719446e-02],
       [-1.4

In [39]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:,2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18*i:18*(i+1), :].reshape(1,-1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j]!=0:
            test_x[i][j] = (test_x[i][j] - mean_x[j])/ std_x[j]
            
test_x = np.concatenate((np.ones([240,1]), test_x), axis=1).astype(float)
test_x

<ipython-input-39-78c4c6eca95b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data == 'NR'] = 0


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [40]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 6.41522970e+00],
       [ 1.82649484e+01],
       [ 2.22241749e+01],
       [ 8.88388397e+00],
       [ 2.71933762e+01],
       [ 2.13455627e+01],
       [ 2.28286546e+01],
       [ 3.10503047e+01],
       [ 1.58299003e+01],
       [ 6.13340786e+01],
       [ 1.48760539e+01],
       [ 1.19790557e+01],
       [ 6.67789255e+01],
       [ 5.08330199e+01],
       [ 2.04407213e+01],
       [ 1.12550029e+01],
       [ 3.18449144e+01],
       [ 6.83607734e+01],
       [-1.46989747e+00],
       [ 1.80064086e+01],
       [ 4.02201540e+01],
       [ 7.21608434e+01],
       [ 9.13462931e+00],
       [ 1.82703391e+01],
       [ 1.52300935e+01],
       [ 3.68305050e+01],
       [ 1.65491168e+01],
       [ 7.38121234e+01],
       [ 9.66140349e+00],
       [ 5.53966303e+01],
       [ 2.41947153e+01],
       [ 6.69710906e+00],
       [ 2.96999846e+00],
       [ 2.23570975e+01],
       [ 2.92177072e+01],
       [ 3.83624112e+01],
       [ 4.44750343e+01],
       [ 3.02296986e+01],
       [ 4.1

In [41]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    csv_writer.writerow(header)
    
    for i in range(len(ans_y)):
        row = ['id_'+str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id_0', 6.415229695638989]
['id_1', 18.26494837737162]
['id_2', 22.224174890928943]
['id_3', 8.883883966978946]
['id_4', 27.193376164609266]
['id_5', 21.3455627035276]
['id_6', 22.828654637008164]
['id_7', 31.050304747567044]
['id_8', 15.829900298352289]
['id_9', 61.334078619842366]
['id_10', 14.876053923962456]
['id_11', 11.979055705565873]
['id_12', 66.77892550659934]
['id_13', 50.833019923671884]
['id_14', 20.44072132844689]
['id_15', 11.255002856843465]
['id_16', 31.84491437778479]
['id_17', 68.36077340583554]
['id_18', -1.4698974688808637]
['id_19', 18.006408564847465]
['id_20', 40.22015403233897]
['id_21', 72.16084339324183]
['id_22', 9.13462930611351]
['id_23', 18.27033914813266]
['id_24', 15.23009346330601]
['id_25', 36.8305050083011]
['id_26', 16.549116778913334]
['id_27', 73.81212343098493]
['id_28', 9.661403493925306]
['id_29', 55.39663027463545]
['id_30', 24.19471525001171]
['id_31', 6.697109055150166]
['id_32', 2.969998460292375]
['id_33', 22.35709752860876]
['id_34', 29.